In [91]:
import folium
import wget
import pandas as pd

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

from math import sin, cos, sqrt, atan2, radians

## 1. Mark all launch sites on map

In [92]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [93]:
# look at coordinates for each site in a dataframe
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [94]:
# create folium map object
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [95]:
# use folium.Circle to highlight circle area with text label on specific coordinate
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [96]:
for index, i in launch_sites_df.iterrows():
    lat = i['Lat']
    long = i['Long']
    print([lat, long])

[28.56230197, -80.57735648]
[28.56319718, -80.57682003]
[28.57325457, -80.64689529]
[34.63283416, -120.6107455]


In [97]:
# add a circle for each launch site
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
circles = []
markers = []
for index, i in launch_sites_df.iterrows():
    coord = [i['Lat'], i['Long']]
    circles = folium.Circle(coord, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(i['Launch Site']))
    markers = folium.map.Marker(
    coord,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % i['Launch Site'],
        )
    )
    site_map.add_child(circles)
    site_map.add_child(markers)
    
site_map

## 2. Mark success/failed launches for each site map

In [98]:
# create markers for all launch records
    ## successful: class = 1 -> green
    ## unsuccessful: class = 0 -> red
    
# because many records will have same coordinate, marker clusters good to simplify map
marker_cluster = MarkerCluster()

In [99]:
spacex_df['marker_color'] = ''
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


In [100]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    coord = [record['Lat'], record['Long']]
    marker = folium.Marker(
    coord,
    # Create an icon as a text label
    icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

## 3. Calculate distances between launch site and its proximities

In [101]:
# create mousposoiton to get coordinate for mouse over a point on map
# add MousePosition to get coordinate (Lat, Long) for a mouse over on the map
formatter = 'function(num) {return L.Util.formatNum(num, 5);};'
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [102]:
# calculate distance between two points
def calculate_distance(coord1, coord2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(coord1[0])
    lon1 = radians(coord1[1])
    lat2 = radians(coord2[0])
    lon2 = radians(coord[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


In [103]:
# find distance to closest coastline
Lat = 28.56367  
Lon = -80.57163
coord = [28.563197,	-80.576820]
beach = [28.56257, -80.56754]
distance_coastline = calculate_distance(coord, beach)
distance_coastline

0.0697411054347723

In [104]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
coordinate = [28.563197,	-80.576820]
beach = [28.56257, -80.56754]
distance_marker = folium.Marker(
    beach,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

lines = folium.PolyLine(locations = [coordinate, beach], weight = 1)

site_map.add_child(lines)
site_map.add_child(distance_marker)

In [105]:
# create a marker and find distance to closest city, railroad, and highway

# set up coordinates and find respective distances
highway = [28.56301, -80.57075]
highway_distance = calculate_distance(coord, highway)

city = [28.6124, -80.81036]
city_distance = calculate_distance(coord, city)

railroad = [28.57865, -80.80563]
railroad_distance = calculate_distance(coord, railroad)

In [106]:
# tbh I'd rather iterate over all this but it's prolly better to hard code it out each time for practice
distance_marker = folium.Marker(
    highway,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(highway_distance),
        )
    )

lines = folium.PolyLine(locations = [coordinate, highway], weight = 1)

site_map.add_child(lines)
site_map.add_child(distance_marker)

distance_marker = folium.Marker(
    city,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(city_distance),
        )
    )

lines = folium.PolyLine(locations = [coordinate, city], weight = 1)

site_map.add_child(lines)
site_map.add_child(distance_marker)

distance_marker = folium.Marker(
    railroad,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(railroad_distance),
        )
    )

lines = folium.PolyLine(locations = [coordinate, railroad], weight = 1)

site_map.add_child(lines)
site_map.add_child(distance_marker)